# agentscope.moudle

agentscope.module 模块提供了一个名为 StateModule 的类，用于支持 嵌套状态的序列化与反序列化。这对于需要保存和恢复复杂对象状态（如智能体、模块、训练状态等）的场景非常有用。 

StateModule 提供以下三个核心方法： 
- register_state(attr_name, custom_to_json=None, custom_from_json=None)  注册一个属性，使其在状态保存/加载时被自动处理。 
- state_dict()  获取当前模块及其嵌套模块的完整状态字典（可 JSON 序列化）。 
- load_state_dict(state_dict, strict=True)  从状态字典中恢复模块状态。 
     

## register_state 

用于将某个属性纳入状态管理。可以指定自定义的序列化/反序列化函数。 

In [1]:
from agentscope.module import StateModule
import json

class MyModule(StateModule):
    def __init__(self):
        super().__init__()
        self.counter = 0
        self.data = {"a": 1, "b": [2, 3]}
        # 注册普通属性（自动使用 json.dumps / json.loads）
        self.register_state("counter")
        self.register_state("data")

    def increment(self):
        self.counter += 1

## state_dict 

获取当前状态字典，包括所有已注册的属性，以及嵌套的 StateModule 子模块。 

In [2]:
module = MyModule()
module.increment()
sd = module.state_dict()
print(json.dumps(sd, indent=2))

{
  "counter": 1,
  "data": {
    "a": 1,
    "b": [
      2,
      3
    ]
  }
}


## load_state_dict 

从状态字典恢复状态： 

In [3]:
new_module = MyModule()
new_module.load_state_dict(sd)
print(new_module.counter)  # 输出: 1

1


StateModule 支持嵌套，即一个 StateModule 中包含另一个 StateModule 实例，状态会自动递归保存/加载。

In [4]:
class SubModule(StateModule):
    def __init__(self, value):
        super().__init__()
        self.value = value
        self.register_state("value")

class MainModule(StateModule):
    def __init__(self):
        super().__init__()
        self.sub = SubModule(42)
        # 注意：不需要显式 register_state("sub")
        # 因为 sub 是 StateModule 子类，会自动被识别并序列化

main = MainModule()
sd = main.state_dict()
print(json.dumps(sd, indent=2))

{
  "sub": {
    "value": 42
  }
}


In [5]:
main2 = MainModule()
main2.load_state_dict(sd)
print(main2.sub.value)  # 输出: 42

42


对于不能直接 JSON 序列化的对象（如 datetime、自定义类等），可提供 custom_to_json 和 custom_from_json。

In [6]:
from datetime import datetime

class TimeModule(StateModule):
    def __init__(self):
        super().__init__()
        self.timestamp = datetime.now()
        self.register_state(
            "timestamp",
            custom_to_json=lambda dt: dt.isoformat(),
            custom_from_json=lambda s: datetime.fromisoformat(s)
        )

tm = TimeModule()
sd = tm.state_dict()
tm2 = TimeModule()
tm2.load_state_dict(sd)
print(tm2.timestamp)  # 恢复后的 datetime 对象

2025-10-02 23:13:44.250312


## strict 参数说明 
strict=True（默认）：如果 state_dict 缺少模块中已注册的 key，会抛出异常。

strict=False：忽略缺失的 key，只加载存在的部分。
     

In [7]:
partial_sd = {"counter": 99}  # 缺少 "data"
module = MyModule()
module.load_state_dict(partial_sd, strict=False)  # 成功，data 保持原值
# module.load_state_dict(partial_sd, strict=True)  # 报错！